In [2]:
import openai
import os
from effectful.handlers.llm import Template
from effectful.handlers.llm.providers import OpenAIAPIProvider, tool_call
from effectful.handlers.llm.structure import DecodeError, decode
from effectful.handlers.llm.synthesis import ProgramSynthesis
from effectful.ops.semantics import fwd, handler
from effectful.ops.syntax import defop

provider = OpenAIAPIProvider(openai.OpenAI(api_key=os.environ['OPENAI_API_KEY']))

In [3]:
@Template.define
def limerick(theme: str) -> str:
    """Write a limerick on the theme of {theme}"""
    raise NotImplementedError

In [5]:
with handler(provider):
    print(limerick("floating point computations"))

In computing, precision's a must,  
Floating points help numbers adjust.  
But just be aware,  
Errors can snare,  
And algorithms sometimes mistrust.


Now we're going to encode some examples from the pocketflow cookbook into the effectful-llm.

### Pocketflow Agent
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-agent)

Idea: a state-machine, with the actions, decide (between searching, answer), search, answer

```mermaid
stateDiagram-v2
    [*] --> Decide
    Decide --> Search : search
    Decide --> Answer : answer
    Search --> Decide
    Answer --> [*]
```

in pocketflow this is encoded using this class node, which states subclass and implement three methods for (`prep`, `exec`, `post`).

`prep` - takes information from a shared context
`exec` - takes this information and feeds it to an LLM call 
`post` - stores the LLM call into the shared context, and returns the next action

For example:
```python
class DecideAction(Node):
    def prep(self, shared):
        context = shared.get('context') or ''
        question = shared['question']
        return question, context
    
    def exec(self, inputs):
        question, context = inputs
        prompt = """ you are a research assistant that can search the web, ... """
        res = call_llm(prompt)
        return decode(res)
    
    def post(self, shared, res):
        shared['context'] = exec_res['answer']
        return result['action']
```


we could encode this in several ways, I'm not sure what is the most convenient. Here's one idealised example:

In [10]:
from typing import Annotated

class DecideAgent:
    """
    Research Assistant that can search the web.
    """
    question: Annotated[str, "Question to find an answer to"]
    __actions__ = {
       'search': "SearchAgent",
       'answer': "AnswerAgent"
    }

class SearchAgent:
    """
    Look up more information on the web.
    """
    query: Annotated[str, "What to search for"]
    __actions__ = {
        'decide': DecideAgent,
        'answer': "AnswerAgent"
    }
class AnswerAgent:
    """
    Based on the information, answer the question
    """
    question: Annotated[str, "Question to find the answer to"]
    research: Annotated[str, "Research that has been done"]


### PocketFlow Async Basic example
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-async-basic)

Same model as above, but using async nodes
```python

class FetchRecipies:
    async def prep(self, shared):
        return shared['ingredient']
    async def exec(self, ingredient):
        return await fetch_recipies(ingredient)
    async def post(self, shared, res):
        shared['recipes'] = res
        return "suggest"
class SuggestRecipe:
    async def prep(self, shared):
        return shared['recipes']
    async def exec(self, recipes):
        suggestion = await call_llm("Choose the best recipe from {','.join(recipes)}")
        return suggestion
    async def post(self, shared, res):
        shared['suggestion'] = suggestion
        return "approve"
class GetApproval:
    async def prep(self, shared): return shared['suggestion']
    async def exec(self, suggestion): return await input('Accept this recipe?')
    async def post(self, shared, answer):
        if answer == "y":
            return "accept"
        else:
            return "retry"
```
This could probably be encoded using the same structure as above. I assume this is trivial, so left out, but should be a similar design to the previous

### Pocketflow BatchFlow example
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-batch-flow).

Introduces the notion of parameters, which allows batching flows over several choices of parameter

```python
class LoadImage:
    def prep(self, shared): return os.path.join("images", self.params['input'])
    def exec(self, image_path): return Image.load(image_path)
    def post(self, shared, res): shared["image"] = res; return "apply_filter"

class ApplyFilter:
    def prep(self, shared): return shared['image'], self.params['filter']
    def exec(self, inputs):
        image, filter_type = inputs
        match filter_type: ...
    def post(self, shared, res):
        shared['filtered_image'] = res
        return "save"
class SaveImage:
    def prep(self, shared): 
        os.makedirs('output')
        input_name, filter_name, output_path = ...
        return shared['filtered_image'], output_path
    def exec(self, inputs): image, output_path = inputs; image.save(output_path, "jpeg"); return output_path
    def post(self, shared, res): return "default"
```

Batching allows taking a single flow and runnning it over a series of parameters
```python
class ImageBatchFlow(BatchFlow):
    def prep(self, shared):
        images = ['cat.jpeg', 'dog.jpeg', 'bird.jpeg']
        filters = ['grayscale', 'blur', 'sepia']
        params = []
        for img in images: for filter in filters: params.append({'input':img, 'filter': f})
        return params
```

I'm tempted to encode this somehow using generics?

In [ ]:
class LoadImage[I,F](Generic[Annotated[I, 'image'], Annotated[F,'filter']]):
    """Load an image from file"""
    __actions__ = {'apply_filter': "ApplyFilter[I,F]"}

### Pocketflow Batch
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-batch)
Demonstrates batching - idea is subclassing batchnode means pre and post need to handle batches, the exec is written as if straight line unbatched

```python
class TranslateTextNode(BatchNode):
    def prep(self, shared):
        text, languages = shared["text"], shared['languages']
        return [(text,lang) for lang in langauges]
    def exec(self, data): text, language = data; return call_llm(f"translate the following into {language}: {text}")
    def post(self, shared, ress):
        for res in ress: os.write(res['language'], res['translation'])
```

### Travel Advisor Chat with Guardrails

Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-chat-guardrail)

```python 
class GuardrailNode:
    def prep(self, shared):
        return shared['user_input']
    def exec(self, user_input):
        return call_llm('evaluate if the following query is travel related ...') 
    def post(self, shared, is_valid):
        if not is_vaild: return "retry"
        shared["messages"].append({'role': "user", "content": shared['user_input']})
        return "process"
```

maybe this could be encoded using handlers? (playing a bit fast and loose with the exact definitions here)

In [ ]:
@Template.define
def travel_query(user_query: str) -> str:
    """Produces a concise (<100) word answer to the query {user_query}"""
    raise NotHandled

@Template.define
def is_safe_query(user_query: str) -> bool:
    """Tests whether the user query {user_query} is related to travel advice"""
    raise NotHandled

def guardrail(user_query: str):
    if not is_safe_query(user_query):
        raise InvalidQuery
    return fwd(user_query)

with handler({validate: is_safe_query}):
    travel_query(user_input)

alternatively using types?

In [ ]:
@Template.define
def is_safe_query(user_query: str) -> bool:
    """Tests whether the user query {user_query} is related to travel advice"""
    raise NotHandled

@Template.define
def travel_query(user_query: Anootated[str, is_safe_query]) -> str:
    """Produces a concise (<100) word answer to the query {user_query}"""
    raise NotHandled

@Template.define
def travel_query(user_query: str):

### PocketFlow Chat with Memory
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-chat-memory)

```python 
class GetUserQuestionNode:
    def prep(self, shared): shared['messages'] = shared.get('messages', [])
    def exec(self, _): return input()
    def post(self, shared, res): shared['messages'].append({'role': 'user', 'content': exec_res}); return "retrieve"

class AnswerNode:
    def prep(self, shared):
        recent_messages = shared.get('messages', [])[-6:]
        context = []
        if (relevant_convos := shared.get('retrieved_conversation')):
            context.append({'role': 'system', 'content': 'the following is a relevant past conversation that might help'})
            context.extend(relevant_convos)
        context.extend(recent_messages)
        context.append({'role': 'system', 'content': 'now continue the conversation'})
        return context
    def exec(self, messages): return call_llm(messages)
    def post(self, shared, res):
        shared['messages'].append({'role': 'assistant', 'content': res})
        if len(shared['messages']) > 6: return 'embed'
        return 'question'

class EmbedNode:
    def prep(self, shared):
        oldest_pair, shared['messages'] = shared['messages'][:2], shared['messages'][2:]
        return oldest_pair
    def exec(conversation):
        embedding = get_embedding('\n'.join(msg['content'] for msg in conversation))
        return {'conversation': conversation, 'embedding': embedding}
    def post(self, shared, res):
        add_vector(shared['vector_index'], (res['embedding']))
        return 'question'

class RetrieveNode:
    def prep(self, shared): return next(msg for msg in shared['messages'] if msg['role'] == 'user', None)
    def exec(self, input): 
        query = inputs['query']; vector_index = inputs['vector_index']; vector_items = inputs['vector_items']
        [index], [dist] = search_vectors(inputs['vector_index'], get_embedding(inputs['query']), k = 1)
        return {'conversation': inputs['vector_items'][index], 'distance': dist}
    def post(self, shared, res):
        shared['retrieved_conversation'] = res['conversation']
        return 'answer'
```

In [ ]:
@Template.define
class ChatAgent:
    messages = []
    def question():
        'ask'

### Pocketflow LLM Streaming
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-llm-streaming)

```python
class StreamNode:
    def prep(self, shared):
        interrupt_evt = threading.Event()
        def wait_for_interrupt(): interrupt_evt.set()
        listener_thread = threading.Thread(target = wait_for_interrupt)
        listener_thread.start()
        chunks = stream_llm(shared['prompt'])
        return chunks, interrupt_evt, listener_thread
    def exec(self, inputs):
        chunks, interrupt_evt, listener_thread = inputs

        for chunk in chunks:
            if interrupt_evt.is_set():
                break
            print(chunk.choices[0].delta.content)
        
        return interrupt_evt, listener_thread
    def post(self, shared, res):
        interrupt_evt, listener_thread = res 
        interrupt_evt.set(); listener_thread.join()
        return 'default'
```

Probably good to support this, though Pocket-flow doesn't really have the nicest interface here...

### Pocketflow Majority vote
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-majority-vote)

illustrates how to use the batching mechanism to implement majority voting

```python
class MajorityVote(BatchNode):
    def prep(self, shared):
        question = shared['question']
        attempts = shared.get('num_tries', 3)
        return [question for _ in range(attempts)]

    def exec(self, question):
        return call_llm("Please answer the user's question below\n{question},answer")

    def post(self, shared, ress):
        answers = [res['answer'] for res in ress]
        best_answer, freq = Counter(answers).most_common(1)[0]
        return 'end'
```

### Pocketflow Multi-Agent Game
Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-multi-agent)

Async communication between two nodes

```python
class AsyncHinter(AsyncNode):
    async def prep(self, shared):
        guess = await shared['hinter_queue'].get()
        if guess == 'GAME_OVER': return None
        return shared['target_word'], shared['forbidden_words'], shared.get('past_guesses', [])
    
    async def exec(self, inputs):
        target, forbidden, past_guesses = inputs
        prompt = 'generate hint for {target}, forbidden words: {forbidden}, past guesses were: {past_guesses}'
        hint = call_llm(prompt)
        return hint
    
    async def post(self, shared, res):
        if res is None:
            return 'end'
        await shared['guesser_queue'].put(exec_res)
        return 'continue'
    
class AsyncGuesser(AsyncNode):
    async def prep(self, shared):
        hint = await shared['guesser_queue'].get()
        return hint, shared.get('past_guesses', [])
    
    async def exec(self, inputs):
        hint, past_guesses = inputs
        prompt = 'given hint {hint}, past guesses {past_guesses}, make a new guess'
        guess = call_llm(prompt)
        return guess
    
    async def post(self, shared, res):
        if res == shared['target_word']:
            await shared['hinter_queue'].put('GAME_OVER')
            return 'end'
        shared['past_guesses'].append(exec_res)
        await shared['hinter_queue'].put(exec_res)
        return 'continue'
        
```

### Pocketflow parallel batch flow

Taken from [here](https://github.com/The-Pocket/PocketFlow/tree/main/cookbook/pocketflow-parallel-batch-flow)

same code as batched, just changes execution strategy such that each flow is run in parallel
```python
class ImageParallelBatchFlow(AsyncParallelBatchFlow):
    
    async def prep_async(self, shared):
        images = shared.get("images",[])
        filters = ['grayscale', 'blur', 'sepia']
        params = []
        for image_path in images:
            for filter_type in filters:
                params.append({ 'image_path': image_path, 'filter': filter_type })
        return params
```

```python
class TranslateTextNodeParallel(AsyncParallelBatchNode):
    async def prep_async(self, shared):
        text = shared.get("text", "no text provided")
        languages = shared.get("langauges", [])
        return [(text,lang) for lang in languages]
    async def exec_async(self, inputs):
        text, language = inputs
        result = await call_llm("please translate the following markdown file into {language}")
        return {"language": language, "translation": result}
    async def post_async(self, shared, res):
        for file in res:
            with open(file['language'], 'w') as f: await f.write(file['translation'])
        return 'default'
```

### Pocketflow RAG example

```python
class ChunkDocumentsNode(BatchNode):
    def prep(self,shared): return shared['texts']
    def exec(self, text): return fixed_size_chunk(text)
    def post(self,shared,res): shared['texts'] = [doc for doc_ls in res for doc in doc_ls]; return 'default'
class EmbedDocumentsNode(BatchNode):
    def prep(self,shared): return shared['texts']
    def exec(self,text): return get_embedding(text)
    def post(self,shared, res): shared['embeddings'] = np.array(res); return 'default'
class CreateIndexNode(Node):
    def prep(self, shared): return shared['query']
    def exec(self,query): return get_embedding(query)
    def post(Self,shared, res): shared['query_embedding'] = res
class RetrieveDocumentsNode(Node):
    def prep(self,shared): return shared['query_embedding'], shared['index'], shared['texts']
    def exec(self,inputs): q_embed, index, texts = inputs; dists, inds = index.search(q_embed, k=1); return {'text': texts[inds[0][0]], 'ind': inds[0][0]}
    def post(self,shared,res): shared['retrieved'] = res; return 'default
```

### Pocketflow Supervisor Flow
Taken from [here](https://github.com/The-Pocket/PocketFlow/blob/main/cookbook/pocketflow-supervisor/flow.py)

```python
class SupervisorNode(Node):
    def prep(self,shared): return shared['answer']
    def exec(self,answer):
        is_nonsense = any(marker in answer for marker in ['coffee break', 'who knows?', 'made up', '42'])
        if is_nonsense: return {'valid': False, 'reason': 'Answer appears to be nonsense'}
        else: return {'valid': True, 'reason': 'Answer appears to be legitimate'}
    def post(self, shared, res):
        if not res['valid']: return 'retry'
```